<h1>Implementação - Processamento Digital de Imagens Aplicado a Identificação Automática de Placas de Trânsito</h1>

Esta é a implementação do projeto "Processamento Digital de Imagens Aplicado a Identificação Automática de Placas de Trânsito", disponível em: http://ojs.unirg.edu.br/index.php/1/article/view/2280

O artigo foi utilizado como referência para o desenvolvimento deste projeto. Foram utilizados métodos diferentes dos descritos no artigo original.

In [1]:
import imageio
import matplotlib.pyplot as plt
import matplotlib.pyplot as pp
import matplotlib.patches as patches
import numpy as np
import cv2
import os
import math

from collections import deque
from skimage import data
from skimage.color import rgb2hsv
from skimage import filters
from skimage import exposure
from skimage import color, img_as_float
from skimage import measure

<h1>Primeira etapa</h1>

Ao  início  do  algoritmo,  é  lida  a  imagem  de  entrada  (no  espaço  de  cores  padrão RGB), em que se quer verificar a existência de uma placa de parada obrigatória.

In [2]:
path = os.path.join(os.getcwd(), 'Dataset')
print(path)

C:\Users\anaca\Documents\IFCE\PDI\Trabalho final\Dataset


Dentro do dataset, o programa vai pegar uma série de informações:

- caminho: diretório das imagens
- file: uma lista com o nome de todo arquivo que está na pasta Dataset

O segundo 'for' vai pegar cada file

In [3]:
data_set = []
gray_data_set = []
        
for caminho, d, file in os.walk(path):
    for filename in file:
        
        img = cv2.imread(os.path.join(caminho,filename))
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        
        data_set.append(img)
        gray_data_set.append(gray)

<h1>Segunda etapa</h1>

Para  a  detecção  de  uma  placa  de  pare,  primeiramente  é  feita  a  separação  dos objetos  em  vermelho  do  restante  da  imagem. Esta  tarefa  é  realizada  convertendo  a imagem  para  o  espaço  de  cores  HSV (função rgb2hsv). Busca-se  por pixels com valores de matiz (hue) 10% mais próximos às extremidades iniciais e finais do conjunto possível de valores representados nesta banda (a matiz armazena as variações cromáticas  da  imagem  de  entrada  com  valores  compreendidos  entre  0  e 1) e com  um valor  de  saturação acima  de 50% (pois procura-se por cores mais puras). Isto  gera  uma máscara que é aplicada à imagem original separando os objetos com cor predominantemente vermelha dos demais objetos presentes na  imagem.

In [14]:
def red_detection(img):
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    ## Gen lower mask (0-5) and upper mask (175-180) of RED
    mask1 = cv2.inRange(img_hsv, (0,50,20), (5,255,255))
    mask2 = cv2.inRange(img_hsv, (175,50,20), (180,255,255))

    ## Merge the mask and crop the red regions
    mask = cv2.bitwise_or(mask1, mask2 )
    croped = cv2.bitwise_and(img, img, mask=mask)
    
    return mask, croped

Para finalizar, somente são considerados objetos com dimensão de, no mínimo, 10% do tamanho total da imagem. Qualquer objeto detectado, com dimensão inferior a 10% da dimensão da imagem de entrada, é descartado pela chamada da função bwareaopen.

In [5]:
def dimension10(mask,img):
    blur = cv2.blur(mask,(4,4))
    mask = cv2.threshold(blur, 175 , 250, cv2.THRESH_BINARY)
    mask = mask[1]
    croped = cv2.bitwise_and(img, img, mask=mask)
    
    return mask, croped

<h1>Terceira Etapa</h1>

Após segmentar um objeto vermelho da imagem, onde uma possível placa de pare estará,  a  imagem  passa  por  um  processo  de detecção  de  bordas  pela  máscara  de gradiente  binário  nesta  imagem  segmentada,  utilizando  o  filtro  de  Sobel  aplicado  com  o auxílio de funções do software.

In [6]:
def edge(croped):
    edges = cv2.Canny(croped,100,200)
    
    return edges

"plt.subplot(121),plt.imshow(croped,cmap = 'gray')\nplt.title('Original Image'), plt.xticks([]), plt.yticks([])\nplt.subplot(122),plt.imshow(edges,cmap = 'gray')\nplt.title('Edge Image'), plt.xticks([]), plt.yticks([])\nplt.show()"

Depois  da  aplicação  do  filtro  de Sobel,  é  possível  que  bordas  do mesmo  objeto fiquem  desconectadas.  Então  a  imagem  gerada  é dilatada  para  realizar  a  junção  de bordas de um mesmo objeto. Essa dilatação é feita através da função imdilate do Matlab.

In [7]:
def sobel(edges):
    kernel = np.ones((3,3),np.uint8)
    dilate = cv2.dilate(edges,kernel,iterations = 1)
    
    return dilate

"plt.subplot(121),plt.imshow(edges)\nplt.title('Edge Image'), plt.xticks([]), plt.yticks([])\nplt.subplot(122),plt.imshow(dilate)\nplt.title('Edge Dilation'), plt.xticks([]), plt.yticks([])"

<h1>Quarta Etapa</h1>

Como  o  propósito  do  algoritmo  é  detectar  apenas  as  bordas  externas  de  uma possível  placa,  ao  resultado  do  processo  anterior,  é  aplicado outro  operador morfológico para  preencher  o  interior de  objetos conectados  (função imfill), resultando  em  uma imagem  onde todos  os objetos  com  bordas  externas  conectadas  estão  totalmente preenchidos.

In [8]:
def get_seed(image):
    w, h = image.shape
    semente = np.zeros((w, h))
    semente[:, :3] = 255 # Lateral esquerda
    semente[:, (h-3):] = 255 # Lateral direita
    
    return semente

In [9]:
def neighborhood(x, y, w, h):
    lista = deque()
    
    pontos = [(x-1,y), (x+1, y), (x,y-1), (x,y+1),
              (x-1,y+1), (x+1, y+1), (x-1,y-1), (x+1,y-1),
             ]
    for p in pontos:
        if (p[0]>=0 and p[1]>=0 and p[0]<w and p[1]<h):
            lista.append((p[0], p[1]))
            
    return lista

In [10]:
def grow_region(image, reg, epsilon=5):
    w, h = image.shape
    
    fila = deque()
    for x in range(w):
        for y in range(h):
            if reg[x,y]==255:
                fila.append((x,y))
                
    while fila: 
        ponto = fila.popleft()
        x = ponto[0]
        y = ponto[1]

        v_list = neighborhood(x, y, w, h)
        for v in v_list:
            v_x = v[0]
            v_y = v[1]
            if( (reg[v_x][v_y]!=255) and (abs(image[x][y]-image[v_x][v_y])<epsilon)):
                reg[v_x][v_y] = 255
                fila.append((v_x,v_y))
    return reg

Posteriormente,  para  melhorar  a  definição  geométrica  do  objeto  detectado  na imagem, é realizada uma erosão com o auxílio da função imerode, cujo resultado é uma imagem mais bem definida do objeto de interesse.

In [11]:
def close(le):
    kernel = np.ones((4,4),np.uint8)
    closing = cv2.morphologyEx(le, cv2.MORPH_CLOSE, kernel)
    
    cv2.imwrite('closing.jpg',closing)
    return closing

<h1>Quinta Etapa</h1>

Neste  ponto,  espera-se  ter  uma  imagem  binária  com  a  geometria  bem definida  do objeto de interesse (um objeto relativamente grande, na cor vermelha, que não esteja em contato com qualquer extremidade da imagem). Esta imagem é utilizada como entrada no algoritmo de detecção de retas pela transformada de Hough.
Neste algoritmo, o primeiro passo  é  detectar  as  bordas  do  objeto  de entrada  e,  para  isto,  é  utilizado  o  detector  de bordas ótimo de Canny.

Com o objeto segmentado e suas bordas detectadas, é aplicada a transformada de Hough  para  encontrar  as  retas  principais  presentes  na  imagem  da  placa. A partir do resultado da transformada de Hough, são selecionados, no máximo, oito linhas  na  imagem,  o  que  é  suficiente  para  detectar  uma  placa  de  pare.  As  linhas detectadas  com  distância  menor  ou  igual  a cinco  pixels  são  unidas  formando  uma  única linha e os segmentos com tamanho menor que sete pixels são descartados.

As  retas  encontradas  são  salvas  em  uma  estrutura  de  dados  que  armazena as coordenadas dos pontos que determinam o início (em amarelo) e o final (em vermelho) de cada  reta.  A  partir  dessas  informações,  são  exibidas  na  imagem  original  as  retas detectadas que descrevem os limites da placa de pare na imagem original.


In [12]:
def hough_transform(img, x):
    image = cv2.imread('closing.jpg')
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

    kernel_size = 5
    blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size),0)

    low_threshold = 50
    high_threshold = 100
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)

    rho = 1  # distance resolution in pixels of the Hough grid
    theta = np.pi / 180  # angular resolution in radians of the Hough grid
    threshold = 15  # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 30  # minimum number of pixels making up a line
    max_line_gap = 30  # maximum gap in pixels between connectable line segments
    line_image = np.copy(img) * 0  # creating a blank to draw lines on

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(edges, rho, theta, threshold, np.array([]),min_line_length, max_line_gap)

    #try:
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(line_image,(x1,y1),(x2,y2),(0,255,0),5)

    # Draw the lines on the  image
    lines_edges = cv2.addWeighted(image, 0.8, line_image, 1, 0)
    path = os.path.join(os.getcwd(), 'Imagens-tratadas')
    cv2.imwrite(os.path.join(path,'houghlines'+str(x)+'.jpg'),lines_edges)
    
    return line_image

#### Percorre as imagens do dataset, realiza as operações e salva as imagens em uma nova pasta

In [13]:
from tqdm import tqdm

x = 1

for img in tqdm(data_set):    
    mask, croped = red_detection(img)
    mask, croped = dimension10(mask,img)
    edges = edge(croped)
    dilate = sobel(edges)
    
    semente = get_seed(dilate)
    le = grow_region(dilate, semente, epsilon=5)
    
    closing = close(le)
    line_image = hough_transform(img, x)
    x += 1

  0%|                                                                                          | 0/198 [00:00<?, ?it/s]<ipython-input-10-46c31f6ee4df>:19: RuntimeWarning: overflow encountered in ubyte_scalars
  if( (reg[v_x][v_y]!=255) and (abs(image[x][y]-image[v_x][v_y])<epsilon)):
  1%|▊                                                                               | 2/198 [01:10<1:55:08, 35.25s/it]


KeyboardInterrupt: 